In [5]:
import tensorflow as tf

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Ustawienie pierwszego GPU dostępnego dla TensorFlow
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Wyjątek może zostać wyrzucony jeśli lista urządzeń jest modyfikowana po zainicjowaniu przez TensorFlow
    print(e)


1 Physical GPUs, 1 Logical GPU


In [8]:
train_dir = 'data/training/'
valid_dir = 'data/validation/'
test_dir  = 'data/files_for_testing/'

In [15]:
import os

def create_data_loaders(train_dir, valid_dir, test_dir, batch_size, img_size=(224,224)):
    AUTOTUNE = tf.data.experimental.AUTOTUNE

    def preprocess_img(image, label):
        return tf.image.resize(image, img_size), label
    
    train_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
        train_dir,
        shuffle = True,
        batch_size = batch_size,
        image_size = img_size
    )

    valid_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
        valid_dir,
        shuffle = True,
        batch_size = batch_size,
        image_size = img_size
    )

    test_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
        test_dir,
        shuffle = False,
        batch_size = batch_size,
        image_size = img_size
    )

    class_names = train_dataset_raw.class_names

    train_dataset = train_dataset_raw.map(preprocess_img, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE)
    valid_dataset = valid_dataset_raw.map(preprocess_img, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE)
    test_dataset = test_dataset_raw.map(preprocess_img, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE)

    return train_dataset, valid_dataset, test_dataset, class_names



In [17]:
batch_size = 32
train_dataset, valid_dataset, test_dataset, class_names = create_data_loaders(train_dir, valid_dir, test_dir, batch_size)

Found 4352 files belonging to 7 classes.
Found 1159 files belonging to 7 classes.
Found 295 files belonging to 7 classes.


In [18]:
print("Classes:", class_names)

Classes: ['Degas', 'Lorrain', 'Monet', 'Munch', 'Poussin', 'Renoir', 'VanGogh']


In [19]:
def decode(message_file):
    with open(message_file, 'r') as file:
        lines = file.readlines()
    
    number_word_pairs = {}
    for line in lines:
        parts = line.strip().split(' ')
        number_word_pairs[int(parts[0])] = ' '.join(parts[1:])
    
    message_numbers = []
    current_line_length = 1
    current_number = 1
    
    while current_number in number_word_pairs:
        message_numbers.append(current_number)
        current_line_length += 1
        current_number += current_line_length
    
    message_words = [number_word_pairs[num] for num in message_numbers if num in number_word_pairs]
    return ' '.join(message_words)

In [20]:
decoded_message = decode('coding_qual_input.txt')
print(decoded_message)

design all skill whole check deal wish visit now moment offer planet people electric lot huge system card current man way our parent wait
